In [8]:
import matplotlib.pyplot as plt 
from matplotlib.animation import FuncAnimation
import numpy as np
from time import sleep
from math import pi, cos , sin ,exp ,ceil
from IPython import display


N = 200  #number of grid points
cfl = 0.01 
endtime = 18*pi #simulation time
initials = 'sin' #  'sin' or 'e' 
alfa = 0.01 # 1  or 0.01
            
h = 6*pi/N
delT = cfl
t = np.arange(0,endtime+delT,delT)
x = np.linspace(0,6*pi-h,N)
frames = len(t)

S = np.zeros([frames,N])
S2 = np.zeros([frames,N])
if initials == 'e':
    s = np.zeros([N])
    for i in range(N):
            s[i] = exp(-((x[i]-pi)**2)/((2*0.25)**2))
    S[0,:] = s
    S2[0,:] = s
elif initials == 'sin':
    S[0,:] = np.sin(x)
    S2[0,:] = np.sin(x)


#First derivative
M = np.zeros([N,N])
B = np.zeros([N,N])
for i in range(N-1):
    M[i,i+1] = 1
    M[i+1,i] = -1
    B[i,i] = 2/3
    B[i,i+1] = 1/6
    B[i+1,i] = 1/6
M[0,N-1] = -1
M[N-1,0] = 1
B[0,N-1] = 1/6
B[N-1,0] = 1/6
B[N-1,N-1] = 2/3
M= (1/(2*h))*M
G = np.linalg.inv(B)
D1 = G.dot(M)
D2 = D1.dot(D1)



I = np.identity(N)
dummy1 = (I-alfa*0.25*delT*D2)
dummy2 = np.linalg.inv(dummy1)
dummy3 = (I-alfa*0.5*delT*D2)
dummy4 = np.linalg.inv(dummy3)
dummy5 = (I-0.5*delT*D1+alfa*0.25*delT*D2)
dummy6 = (I+alfa*0.5*delT*D2)


dummy32 = (I-alfa*0.25*delT*D2)
dummy42 = np.linalg.inv(dummy32)
dummy62 = (I+alfa*0.25*delT*D2)
for i in range(frames-1):
    u2_1 = dummy2.dot(dummy5).dot(S[i,:])
    u2_2 = dummy42.dot(-0.5*delT*D1.dot(u2_1)+dummy62.dot(S[i,:]))
    u3 = dummy4.dot(-delT*D1.dot(u2_2)+dummy6.dot(S[i,:]))
    S[i+1,:] = dummy4.dot(-(1/6)*delT*D1.dot(S[i,:])-(2/6)*delT*D1.dot(u2_1)-(2/6)*delT*D1.dot(u2_2)-(1/6)*delT*D1.dot(u3)+dummy6.dot(S[i,:]))
    
    u2 = dummy2.dot(dummy5).dot(S2[i,:])
    S2[i+1,:] = dummy4.dot(-delT*D1.dot(u2)+dummy6.dot(S2[i,:]))

    
    

def animate(i):  
    y2 = S2[i,:]
    y = S[i,:]  
    line2.set_data((x,y2))
    line.set_data((x,y))
    plt.legend([line2,line],['RK2','Rk4'])

fig = plt.figure()
line2  = plt.plot([])[0]
lines = plt.plot([])
line = lines[0]
plt.xlim(0, 6*pi)
plt.ylim(-1,1)

anim = FuncAnimation(fig,animate,frames = frames, interval = 30)
video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()